In [1]:
import tensorflow as tf
import numpy as np

print("Tensorflow version: {}".format(tf.__version__))

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensorflow version: 1.11.0


# Control Dependencies and Identity

## Original (Wrong)

In [2]:
with tf.variable_scope("cdi"):
    x = tf.Variable(1.0)
    y = tf.Variable(0.0)
    
    x_plus_1 = tf.assign_add(x, 1)
    
    with tf.control_dependencies([x_plus_1]):
        y = x
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(5):
            print(sess.run(y))

Instructions for updating:
Colocations handled automatically by placer.
1.0
1.0
1.0
1.0
1.0


## Compare (Correct)

`tf.control_dependencies` only change the defined operation (`tf.identity`), the `y = x` is the first assignment, not the operation. `tf.identity` is the operation which returns the same tensor.

In [3]:
with tf.variable_scope("cdi"):
    x = tf.Variable(1.0)
    y = tf.Variable(0.0)
    
    x_plus_1 = tf.assign_add(x, 1)
    
    with tf.control_dependencies([x_plus_1]):
        y = tf.identity(x)   # the only change
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(5):
            print(sess.run(y))

2.0
3.0
4.0
5.0
6.0


# Loop in Tensorflow

In [5]:
i = tf.constant(0)
c = lambda i: tf.less(i, 10)
b = lambda i: tf.add(i, 1)
r = tf.while_loop(c, b, [i])

with tf.Session() as sess:
    print(sess.run(r))

10


In [8]:
with tf.variable_scope("while_loop"):
    index = tf.constant(0)
    sum_all = tf.constant(0)
    
    # loop_vars must be passed into both functions cond and body
    # *_ means all the other parameters
    cond = lambda index, *_: tf.less(index, 10)  
    def cond_worse(index, sum_all):
        return tf.less(index, 10)
    
    # loop_vars must be passed into both functions cond and body
    def body(index, sum_all):
        tmp = tf.add(sum_all, tf.pow(index, 2))
        index += 1  # you have to increament by yourself
        return (index, tmp)
    
    # return the same body with loop_vars
    # r = (index, sum_all)
    r = tf.while_loop(cond=cond, body=body, loop_vars=[index, sum_all])
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(r))

(10, 285)


In [13]:
cond = lambda index, *_: index < 10
print(cond(12, 'a', 10))

False


In [20]:
# 20 sequence in the length of 10
matrix = tf.placeholder(tf.int32, shape=(10, 20), name="input_matrix")
matrix_rows = tf.shape(matrix)[0]
ta = tf.TensorArray(dtype=tf.int32, size=matrix_rows)

init_state = (0, ta)
condition = lambda i, _: i < matrix_rows
body = lambda i, ta: (i + 1, ta.write(i, matrix[i] * 2))
n, ta_final = tf.while_loop(condition, body, init_state)
# get the final result
ta_final_result = ta_final.stack()

# run the graph
with tf.Session() as sess:
    # print the output of ta_final_result
    print(sess.run(ta_final_result, feed_dict={matrix: np.ones(shape=(10,20), dtype=np.int32)}))

[[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]]


# tf.clip_by_value()

Narrow the value in the array to [min, max]. The value is min if it is smaller than the min value. The value is max if it is bigger than the max value. The value is the same if it is bigger than the min value, but is smaller then the max value.

In [58]:
with tf.variable_scope("clip") as scope:
    scope.reuse_variables()
    int_val = tf.get_variable("intval", [10], initializer=tf.random_normal_initializer(stddev=2.0))
    int_val_clip = tf.clip_by_value(int_val, 0, 2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(int_val))
    print(sess.run(int_val_clip))

[ 2.426633    0.3751102   2.6840754   0.66835743  0.0943675   2.0987873
 -3.2778037  -2.5711484   0.5976639   0.90391505]
[2.         0.3751102  2.         0.66835743 0.0943675  2.
 0.         0.         0.5976639  0.90391505]


In [61]:
(0.3751102 - (-3.2778037)) / (2.426633 - (-3.2778037)) 

0.6403636488770224

# Coordinator

In [40]:
def Coord():
    q = tf.FIFOQueue(10, "float")
    counter = tf.Variable(0.0)
    increment_op = tf.assign_add(counter, tf.constant(1.0))

    enqueue_op = q.enqueue([counter])
    qr = tf.train.QueueRunner(q, enqueue_ops=[increment_op, enqueue_op] * 1)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # coordinator
        crd = tf.train.Coordinator()
        
        # start to add value (a thread) and add it into queue (another thread)
        enqueue_threads = qr.create_threads(sess, coord=crd, start=True)

        # start to fetch value from queue
        for i in range(10):
            print("De: {}".format(sess.run(q.dequeue())))

        # stop 
        crd.request_stop()
        crd.join(enqueue_threads)
        
Coord()

De: 13.0
De: 28.0
De: 41.0
De: 54.0
De: 68.0
De: 87.0
De: 107.0
De: 125.0
De: 139.0
De: 159.0


# concat vs. stack 

In [50]:
with tf.variable_scope("concat") as scope:
    #ar1 = tf.random_normal([1,1,224,224])
    scope.reuse_variables()
    ar3 = tf.get_variable("ar3", [1,1,224,224], initializer=tf.random_normal_initializer(stddev=(0.1)))
    ar4 = tf.get_variable("ar4", [1,1,224,224], initializer=tf.random_normal_initializer(stddev=(0.1)))
    ar5 = tf.concat([ar3,ar4], axis=1)
    sh = tf.shape(ar5)
    
    ar3_1 = tf.squeeze(ar3, [0,1])
    ar4_1 = tf.squeeze(ar4, [0,1])
    stk = tf.expand_dims(tf.stack([ar3_1,ar4_1]), 0)
    stk_shape = tf.shape(stk)

In [51]:
with tf.Session() as sess:
    print(sess.run(tf.shape(ar1)))
    print(sess.run(tf.shape(ar2)))
    print(sess.run(sh))
    print(sess.run(stk_shape))

[  1   1 224 224]
[  1   1 224 224]
[  1   2 224 224]
[  1   2 224 224]


# tf.tile

In [4]:
with tf.variable_scope("tf_tile"):
    data = tf.random_uniform([2,2], minval=0, maxval=5, dtype=tf.int32)

with tf.Session() as sess:
    oriData, tileData = sess.run([data, tf.tile(data, [1,2])])
    print(oriData, "\n", tileData)

[[1 0]
 [2 1]] 
 [[1 0 1 0]
 [2 1 2 1]]


# Reduced Operations

## reduced_sum

In [73]:
with tf.variable_scope("reduced_sum") as scope:
    sum_array = tf.constant([[[1,1,1], [2,2,2], [10,10,10]], [[20,20,20], [100,100,100], [200,200,200]]])
    axis_0 = tf.reduce_sum(sum_array, 0)
    axis_1 = tf.reduce_sum(sum_array, 1)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(sum_array))
    print(sess.run(sum_array).shape)
    print()
    print(sess.run(tf.reduce_sum(sum_array)))
    print()
    print(sess.run(axis_0))
    print(sess.run(axis_0).shape)
    print()
    print(sess.run(axis_1))
    print(sess.run(axis_1).shape)

[[[  1   1   1]
  [  2   2   2]
  [ 10  10  10]]

 [[ 20  20  20]
  [100 100 100]
  [200 200 200]]]
(2, 3, 3)

999

[[ 21  21  21]
 [102 102 102]
 [210 210 210]]
(3, 3)

[[ 13  13  13]
 [320 320 320]]
(2, 3)


## reduced_mean

In [75]:
with tf.variable_scope("reduced_mean") as scope:
    mean_array = tf.constant([[[1,1,1], [2,2,2], [10,10,10]], [[20,20,20], [100,100,100], [200,200,200]]])
    axis_0 = tf.reduce_mean(mean_array, 0)
    axis_1 = tf.reduce_mean(mean_array, 1)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(mean_array))
    print(sess.run(mean_array).shape)
    print()
    print(sess.run(tf.reduce_mean(sum_array)))
    print()
    print(sess.run(axis_0))
    print(sess.run(axis_0).shape)
    print()
    print(sess.run(axis_1))
    print(sess.run(axis_1).shape)

[[[  1   1   1]
  [  2   2   2]
  [ 10  10  10]]

 [[ 20  20  20]
  [100 100 100]
  [200 200 200]]]
(2, 3, 3)

55

[[ 10  10  10]
 [ 51  51  51]
 [105 105 105]]
(3, 3)

[[  4   4   4]
 [106 106 106]]
(2, 3)


# tf.Tensorarray

In [23]:
import tensorflow as tf

with tf.variable_scope("allloc"):
    data = tf.random_uniform([1,4], minval=0, maxval=10, dtype=tf.int32)
    sorted_ut, free_list = tf.nn.top_k(-1 * data, 4)
    
    data2 = tf.random_uniform([1,4], minval=100, maxval=200, dtype=tf.int32)
    empty_at = tf.TensorArray(tf.int32, 4)
    full_at = empty_at.scatter(
            tf.reshape(free_list, (-1,)), 
            tf.reshape(data2, (-1,)))
    a_t = full_at.stack()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    d, s, f, d2, a_t = sess.run([data, sorted_ut, free_list, data2, a_t])
    print("data: {}".format(d))
    print("sorted: {}, reversed: {}".format(s,s*(-1)))
    print("free: {}".format(f))
    print("data2: {}".format(d2))
    print("full_at: {}".format(a_t))

data: [[1 7 0 5]]
sorted: [[ 0 -1 -5 -7]], reversed: [[0 1 5 7]]
free: [[2 0 3 1]]
data2: [[108 170 171 110]]
full_at: [170 110 108 171]


In [66]:
with tf.variable_scope("tensorarray"):
    values = tf.random_normal([10])
    
    index = tf.constant(0)
    tmp = tf.constant(0.0)
    
    values_array = tf.TensorArray(tf.float32, 10)
    values_array = values_array.unstack(values)
    final_array = tf.TensorArray(tf.float32, 10)
    
    cond = lambda index, *_: tf.less(index, 10)
    def body(index, values_array, final_array, tmp):
        current_value = values_array.read(index)
        tmp = current_value + tf.cast(tf.pow(index, 2), dtype=tf.float32)
        final_array = final_array.write(index, tmp)
        index += 1
        return (index, values_array, final_array, tmp)
    
    _, _, fin, _ = tf.while_loop(cond=cond, body=body, loop_vars=[index, values_array, final_array, tmp])
    
    fin = fin.stack()
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    val, finarry = sess.run([values, fin])
    print(val, "\n", finarry)

[-0.50065833 -0.8159259   0.86088264  2.7285786  -1.900892   -0.8135131
 -0.5319745   0.17028719  1.9458723  -0.31174654] 
 [-0.50065833  0.1840741   4.8608828  11.728579   14.099108   24.186487
 35.468025   49.170288   65.94587    80.688255  ]


In [68]:
with tf.variable_scope("tensorarray2"):
    values = tf.random_normal([10])
    values_array = tf.TensorArray(tf.float32, 10)
    values_array = values_array.unstack(values)
    values_array_transform = values_array.stack()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    values, values_array_transform = sess.run([values, values_array_transform])
    print(values, "\n", values_array_transform)

[-0.44759494  0.68972546  1.712836   -1.773806    1.0969163  -0.26527986
 -0.62892115  0.21904905  0.5325642  -1.0592977 ] 
 [-0.44759494  0.68972546  1.712836   -1.773806    1.0969163  -0.26527986
 -0.62892115  0.21904905  0.5325642  -1.0592977 ]


# tf.get_collection

In [2]:
with tf.Graph().as_default():
  with tf.variable_scope("pn"):
    a = tf.constant(0, dtype='float32', name="va")
    b = tf.constant(1, dtype='float32', name="vb")
    c = tf.get_variable("vc", shape=[2, 3], initializer=tf.random_normal_initializer(10.0, 2.0))
    
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      print(sess.run([a+b]))
  
  ns = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="pn")
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    c_, ns_ = sess.run([c, ns])
    print(c_)
    print(ns_)

[1.0]
[[ 4.365282  9.835556 11.242051]
 [13.176279  5.793268 10.558922]]
[array([[ 4.365282,  9.835556, 11.242051],
       [13.176279,  5.793268, 10.558922]], dtype=float32)]
